<a href="https://colab.research.google.com/github/zainab-hsini/TravelTunes/blob/main/TravelTunes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We're going to be building a program that creates a playlist based on the duration of your trip. The user inputs their location and a point X where they wish to go, and the program will create a playlist, based on their preferred genre, that matches the duration of the trip.

We'll be using two APIs:

*   An API to calculate the duration of the trip from point A to point B (TravelTime API)
*   An API that generates a playlist based on some preferences that is the same duration of the trip (Spotify Get Track API)




In [29]:
#set up API keys
from google.colab import userdata

spotify_api_key = userdata.get('Spotify')
triptime_api_key = userdata.get('TripTime')

#handling Spotify requests
import requests

CLIENT_ID = '6e520222634c467da6b12d66950837b3'
CLIENT_SECRET = 'b049e33ef609493cb6b6677084f6c1d4'

AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

We have set up both keys and handled Spotify requests. Now it's time to calculate the trip's duration.

In [30]:
# Installing the necessary packages
!pip install asyncio -q
!pip install nest_asyncio -q
!pip install traveltimepy -q

# Importing necessary packages
import asyncio
import nest_asyncio
from traveltimepy import Location, Coordinates, Transportation, TravelTimeSdk

# Allows nested async loops
nest_asyncio.apply()

# Async function to calculate travel time
async def main():

    # Handling Travel Time requests
    sdk = TravelTimeSdk("8c1fc92c", "5fc314f3af7748628cc293d3e703067c")

    # Define the locations with test coordinates
    departure_location = Location(id="departure_location", coords=Coordinates(lat=departure_lat, lng=departure_lng))
    arrival_location = Location(id="arrival_location", coords=Coordinates(lat=arrival_lat, lng=arrival_lng))

    # Perform the time filter search
    results = await sdk.time_filter_fast_async(
        locations=[departure_location, arrival_location],
        search_ids={
            "departure_location": ["arrival_location"]
        },
        transportation=Transportation(type="driving"),
        one_to_many=True
    )

    trip_duration = None

    # Extract the travel time
    if isinstance(results, list) and len(results) > 0:
        for result in results:
            if result.search_id == 'departure_location':
                for loc in result.locations:
                    if loc.id == 'arrival_location':
                        trip_duration = loc.properties.travel_time
    else:
        print("No valid results found. Please check coordinates and transportation mode.")

    return trip_duration

# Function to get validated float input
def get_validated_float(prompt, min_value, max_value):
    while True:
        try:
            value = float(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print(f"Value out of range. Please enter a value between {min_value} and {max_value}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Get user input for coordinates
print("Welcome to your playlist maker! Let's make your trip a little more fun :)")
print("Where are you? Enter your coordinates below. ")

departure_lat = get_validated_float("Departure latitude: ", -90, 90)
departure_lng = get_validated_float("Departure longitude: ", -180, 180)
print("And where are you going? Enter the coordinates here: ")
arrival_lat = get_validated_float("Arrival latitude: ", -90, 90)
arrival_lng = get_validated_float("Arrival longitude: ", -180, 180)

# Run the async function and get the trip duration
trip_duration = asyncio.run(main())
print("Great! Let's find you a playlist.")

# Coordinates to use for demo:
# The London Eye: 51.5033 (latitude) -0.1196 (longitude)
# Buckingham Palace: 51.5014 (latitude) -0.1419 (longitude)

Welcome to your playlist maker! Let's make your trip a little more fun :)
Where are you? Enter your coordinates below. 
Departure latitude: 51.5033
Departure longitude: -0.1196
And where are you going? Enter the coordinates here: 
Arrival latitude: 51.5014
Arrival longitude: -0.1419
Great! Let's find you a playlist.


After getting the coordinates from the user and calculating the duration of the trip, it is time to get the user's music preferences and create a playlist they can listen to during the ride!

In [31]:
import requests

# Define headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Define supported genres
GENRE_MAP = {
    "Pop": "pop", "Rock": "rock", "Hip-Hop": "hip-hop", "Jazz": "jazz", "Classical": "classical",
    "Country": "country", "Electronic": "electronic", "Reggae": "reggae", "Metal": "metal", "Blues": "blues",
    "Folk": "folk", "Soul": "soul", "Funk": "funk", "Punk": "punk", "Indie": "indie",
    "Alternative": "alternative", "Latin": "latin", "R&B": "r&b", "Dance": "dance", "House": "house"
}

# Print available genres
def print_available_genres():
    print("What would you like to listen to? Choose a genre from our selection below!")
    for i, genre in enumerate(GENRE_MAP.keys(), 1):
        print(f"{i}. {genre}")

# Get recommendations by genre
def get_recommendations_by_genre(genre, limit=50):
    api_genre = GENRE_MAP.get(genre)
    if not api_genre:
        return None

    RECOMMENDATIONS_URL = 'https://api.spotify.com/v1/recommendations'

    params = {
        'seed_genres': api_genre,
        'limit': limit,
        'market': 'US',
    }

    response = requests.get(RECOMMENDATIONS_URL, headers=headers, params=params)
    return response.json()

# Calculate playlist duration
def calculate_playlist(tracks, target_duration_sec):
    selected_tracks = []
    selected_track_ids = set()
    total_duration_sec = 0

    for track in tracks:
        track_id = track['id']
        if track_id in selected_track_ids:
            continue

        track_name = track['name']
        track_duration_ms = track['duration_ms']
        track_duration_sec = track_duration_ms / 1000
        artist_names = ', '.join(artist['name'] for artist in track['artists'])

        if total_duration_sec + track_duration_sec <= target_duration_sec:
            selected_tracks.append((track_name, artist_names))
            selected_track_ids.add(track_id)
            total_duration_sec += track_duration_sec

        if total_duration_sec > target_duration_sec:
            break

    return selected_tracks, total_duration

# Main function
def main():
    print_available_genres()

    while True:
        # Input genre from user
        user_genre = input("Enter your preferred genre: ").strip().title()

        # Fetch recommended tracks based on the genre
        recommendations_data = get_recommendations_by_genre(user_genre, limit=50)
        if recommendations_data:
            tracks = recommendations_data['tracks']

            # Calculate playlist duration
            selected_tracks, total_duration = calculate_playlist(tracks, trip_duration)

            # Output the result
            print("Here's our recommended playlist :)")
            for track_name, artist_names in selected_tracks:
                print(f"- {track_name} by {artist_names}")
            break
        else:
            print("Invalid input, please try again with a valid genre.")

if __name__ == "__main__":
    main()

What would you like to listen to? Choose a genre from our selection below!
1. Pop
2. Rock
3. Hip-Hop
4. Jazz
5. Classical
6. Country
7. Electronic
8. Reggae
9. Metal
10. Blues
11. Folk
12. Soul
13. Funk
14. Punk
15. Indie
16. Alternative
17. Latin
18. R&B
19. Dance
20. House
Enter your preferred genre: punk
Here's our recommended playlist :)
- Love and Leaving by Julie & The Wrong Guys
- Search And Destroy - LA & Detroit Rehearsals, 1973 by The Stooges
- I Won’t Say I’m Sorry by Face To Face
- Stormy Weather by Arborview
